In [ ]:
## Load Dataframe
import pandas as pd
import os

df_arr = []
root = ".dataDir"
for path, subdirs, files in os.walk(root):
    for name in files:
        df = pd.read_csv(os.path.join(path, name))
        df_arr.append(df)

df_cons = pd.concat(df_arr, ignore_index=True)
df_cons.head()

In [ ]:
## Run Aggregations
df = df_cons.groupby(['ds','tenant','pod']).agg({'dataScannedInGB': 'sum', 'processingTimeinHrs':'sum'})
df['AvgDataInGBperdayByDw']=df.dataScannedInGB*24/df.processingTimeinHrs
df = df.add_suffix('_Count').reset_index()
df.describe(percentiles=[0.25, 0.75])

In [ ]:
## Create Plots
import plotly.express as px
fig = px.box(df, x='ds', y='AvgDataInGBperdayByDw_Count', color='ds', points=False, log_y=True)
fig.update_layout(
    font=dict(family="Didact Gothic"),
    yaxis_title="<b>GB/day/dw<b>",
    xaxis_title="<b>fileFormat<b>",
    title_x=0.5,
    title_font=dict(size=24,color='black'),
    plot_bgcolor='rgba(0, 0, 0, 0)', #'white',  
    paper_bgcolor='rgba(0, 0, 0, 0)', #'white',
    width=1200,  
    height=900
)
fig.update_traces(quartilemethod="exclusive")
fig.update_yaxes(nticks=10, minor=dict(showgrid=True, gridwidth=2, gridcolor='Black'))
fig.show()